# imports

In [1]:
# %matplotlib inline

###KAUFMAN LAB LIBRARY
klib_version = '1.1'

import sys
sys.path.append('../../Analysis_code_Yb/klab_python_lib_Yb/')

import klib
if klib.__version__ != klib_version:
    raise ValueError('Incorrect klib version number.')

from klib.imports import *
from klib.analysis import *
from klib.plotutil import *
from klib.motimage2D import *
from klib.motimage3D import *
from klib.Yb_constants import *
from klib.zynq_client import *
from klib.expPlotter import *

from skimage import io
import matplotlib.pyplot as plt

import numpy as np
%matplotlib inline
import tqdm
from scipy.optimize import curve_fit
from skimage import feature

import scipy.constants as sc
from skimage import data
import glob
from natsort import natsorted
#from vimba import *
import importlib
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy.special import jv


kB=sc.k #boltzmann cont
pi=np.pi
c=sc.c
h=sc.h
g=sc.g
eps0 = sc.epsilon_0
au=sc.physical_constants['atomic mass constant'][0]
dataAddress = Path('../../Yb_data/')
from scipy.linalg import expm

In [2]:
import plotly.graph_objects as go
import plotly.express as px
from plotly import subplots 
import plotly.colors as plotlycolors
import pandas as pd
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, State
from dash.exceptions import PreventUpdate

import re
import copy

cols = plotlycolors.DEFAULT_PLOTLY_COLORS

# test data set

In [2]:
run = 46
exp = ExpFile(dataAddress / 'Yb_230308/Raw Data', run)

..\..\Yb_data\Yb_230308\Raw Data\data_46.h5


In [4]:
expPlotter = ExpPlotter(exp)

In [7]:
plotTTLChannels = ['op','rb1','rb2','rb3','imageshutter','cameratrig','clock']
plotDACChannels = ['slmdepth', 'x1coil', 'y1coil', 'z1coil', 'clockpiezohor']
plotDDSChannels = ['dds0', 'dds4', 'dds8']
expPlotter.setTTLChannels(plotTTLChannels)
expPlotter.setDACChannels(plotDACChannels)
expPlotter.setDDSChannels(plotDDSChannels)
runExpViewer(expPlotter)

Dash app running on http://127.0.0.1:8050/


# experiment plotter class

In [4]:
class Snapshot:
    def __init__(self, t, TTLChannels, DACChannels, DDSFreqChannels, DDSAmpChannels):
        self.t = t
        self.TTLChannels = TTLChannels
        self.DACChannels = DACChannels
        self.DDSFreqChannels = DDSFreqChannels
        self.DDSAmpChannels = DDSAmpChannels

In [5]:
class ExperimentPlotter:
    
    def __init__(self, exp):
        self.exp = exp
        self.variables = self.getVariables()
        self.lines = self.getMasterScriptLines()
        self.TTLChannels = self.getTTLChannels()
        self.DACChannels = self.getDACChannels()
        self.DDSFreqChannels, self.DDSAmpChannels = self.getDDSChannels()
        self.snapshots = []
        self.getSnapshots()
        self.DACData = self.getDACData()
        self.DDSFreqData = self.getDDSFreqData()
        self.DDSAmpData = self.getDDSAmpData()
        self.TTLData = self.getTTLData()
        self.key_name = exp.key_name
        self.key = exp.key
        self.pics = exp.pics
        self.reps = exp.reps
        self.experiment_time = exp.experiment_time
        self.experiment_date = exp.experiment_date
        self.data_addr = exp.data_addr
        self.file_id = exp.file_id
        
    def getMasterScriptLines(self):
        mscript = self.exp.f['Master-Parameters']['Master-Script']
        lines = []
        line = []
        for x in mscript:
            c = x.decode('UTF-8')
            if (c == '\n'):
                lines.append("".join(line))
                line = []
            else:
                line += c
        linesUnc = list(map(self.cleanLines, lines))
        linesFil = list(filter(self.filterEmptyLines, linesUnc))
        linesEvalDAC = list(map(self.evaluateDACVariables, linesFil))
        linesEvalDDS = list(map(self.evaluateDDSVariables, linesEvalDAC))
        linesEvalTime = list(map(self.evaluateTimeVariables, linesEvalDDS))
        linesEvalRepeats = self.evaluateRepeats(linesEvalTime)
        linesEvalPulses = self.evaluatePulses(linesEvalRepeats)
        return linesEvalPulses
    
    def getVariables(self):
        variables = {}
        for key, val in self.exp.f['Master-Parameters']['Variables'].items():
            variables[key] = val[0]
        return variables
        
    def cleanLines(self, line):
        line = line.split('%')[0]
        line = line.split('\r')[0]
        return line

    def filterEmptyLines(self, line):
        if (line == ''):
            return False
        else:
            return True
    
    def evaluateRepeats(self, lines):
        repeatLinesAll = []
        inds = [-1]
        repeatLine = -1
        endLine = -1
        for i, line in enumerate(lines):
            repeatMatch = re.search("^repeat:\s+[0-9]+", line)
            endMatch = re.search("^end\s*", line)
            if (repeatMatch):
                lineSplit = re.split('\s+', repeatMatch[0])
                numRepeats = int(lineSplit[1])
                repeatLine = i
            if (endMatch and repeatLine != -1):
                endLine = i
            if (endLine != -1):
                repeatLines = lines[repeatLine+1:endLine]
                repeatLines = repeatLines*numRepeats
                repeatLinesAll.append(repeatLines)
                inds.append(repeatLine)
                inds.append(endLine)
                endLine = -1
                repeatLine = -1
        linesPieces = [lines[inds[2*j]+1:inds[2*j+1]] + repeatLinesAll[j] for j in range(len(repeatLinesAll))] + [lines[inds[-1]+1:]]
        lines = [line for linesPiece in linesPieces for line in linesPiece]
        return lines
        
    def evaluateDACVariables(self, line):
        dacMatch = re.search("^dac:\s+[\-a-zA-Z0-9_]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
        dacRampMatch = re.search("^dacramp:\s+[\-a-zA-Z0-9_]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)\*\/]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
        if (dacMatch):
            lineSplit = re.split('\s+', dacMatch[0])
            startSplit = re.split('(\+|\-|\*|\/|\(|\))', lineSplit[2])
            for var, val in self.variables.items():
                startSplit = [re.sub("^"+var+"$", str(val), s) for s in startSplit]
            lineSplit[2] = "".join(startSplit)
            lineSplit[2] = str(eval(lineSplit[2]))
            line = " ".join(lineSplit)
        elif (dacRampMatch):
            lineSplit = re.split('\s+', dacRampMatch[0])
            startSplit = re.split('(\+|\-|\*|\/|\(|\))', lineSplit[2])
            endSplit = re.split('(\+|\-|\*|\/|\(|\))', lineSplit[3])
            rampSplit = re.split('(\+|\-|\*|\/|\(|\))', lineSplit[4])
            for var, val in self.variables.items():
                startSplit = [re.sub("^"+var+"$", str(val), s) for s in startSplit]
                endSplit = [re.sub("^"+var+"$", str(val), s) for s in endSplit]
                rampSplit = [re.sub("^"+var+"$", str(val), s) for s in rampSplit]
            lineSplit[2] = "".join(startSplit)
            lineSplit[3] = "".join(endSplit)
            lineSplit[4] = "".join(rampSplit)
            lineSplit[2] = str(eval(lineSplit[2]))
            lineSplit[3] = str(eval(lineSplit[3]))
            lineSplit[4] = str(eval(lineSplit[4]))
            line = " ".join(lineSplit)
        return line
    
    def evaluateDDSVariables(self, line):
        ddsampMatch = re.search("^ddsamp:\s+[\-a-zA-Z0-9_]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
        ddsfreqMatch = re.search("^ddsfreq:\s+[\-a-zA-Z0-9_]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
        ddsRampAmpMatch = re.search("^ddsrampamp:\s+[\-a-zA-Z0-9_]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)\*\/]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
        ddsRampFreqMatch = re.search("^ddsrampfreq:\s+[\-a-zA-Z0-9_]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)\*\/]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
        if (ddsampMatch or ddsfreqMatch):
            if (ddsampMatch):
                ddsMatch = ddsampMatch
            else:
                ddsMatch = ddsfreqMatch
            lineSplit = re.split('\s+', ddsMatch[0])
            startSplit = re.split('(\+|\-|\*|\/|\(|\))', lineSplit[2])
            for var, val in self.variables.items():
                startSplit = [re.sub("^"+var+"$", str(val), s) for s in startSplit]
            lineSplit[2] = "".join(startSplit)
            lineSplit[2] = str(eval(lineSplit[2]))
            line = " ".join(lineSplit)
        elif (ddsRampAmpMatch or ddsRampFreqMatch):
            if (ddsRampAmpMatch):
                ddsRampMatch = ddsRampAmpMatch
            else:
                ddsRampMatch = ddsRampFreqMatch
            lineSplit = re.split('\s+', ddsRampMatch[0])
            startSplit = re.split('(\+|\-|\*|\/|\(|\))', lineSplit[2])
            endSplit = re.split('(\+|\-|\*|\/|\(|\))', lineSplit[3])
            rampSplit = re.split('(\+|\-|\*|\/|\(|\))', lineSplit[4])
            for var, val in self.variables.items():
                startSplit = [re.sub("^"+var+"$", str(val), s) for s in startSplit]
                endSplit = [re.sub("^"+var+"$", str(val), s) for s in endSplit]
                rampSplit = [re.sub("^"+var+"$", str(val), s) for s in rampSplit]
            lineSplit[2] = "".join(startSplit)
            lineSplit[3] = "".join(endSplit)
            lineSplit[4] = "".join(rampSplit)
            lineSplit[2] = str(eval(lineSplit[2]))
            lineSplit[3] = str(eval(lineSplit[3]))
            lineSplit[4] = str(eval(lineSplit[4]))
            line = " ".join(lineSplit)
        return line
    
    def evaluateTimeVariables(self, line):
        tMatch = re.search("^t\s+[\+=]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
        if (tMatch):
            lineSplit = re.split('\s+', tMatch[0])
            timeSplit = re.split('(\+|\-|\*|\/|\(|\))', lineSplit[2])
            for var, val in self.variables.items():
                timeSplit = [re.sub("^"+var+"$", str(val), s) for s in timeSplit]
            lineSplit[2] = "".join(timeSplit)
            lineSplit[2] = str(eval(lineSplit[2]))
            line = " ".join(lineSplit)
        return line
    
    def evaluatePulses(self, lines):
        pulseLinesAll = []
        inds = [-1]
        for i, line in enumerate(lines):
            pulseonMatch = re.search("^pulseon:\s+[\-a-zA-Z0-9_]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
            if (pulseonMatch):
                lineSplit = re.split('\s+', pulseonMatch[0])
                timeSplit = re.split('(\+|\-|\*|\/|\(|\))', lineSplit[2])
                for var, val in self.variables.items():
                    timeSplit = [re.sub("^"+var+"$", str(val), s) for s in timeSplit]
                lineSplit[2] = "".join(timeSplit)
                pulseTime = str(eval(lineSplit[2]))
                pulseLines = ['on: '+lineSplit[1], 't += '+pulseTime, 'off: '+lineSplit[1]]
                pulseLinesAll.append(pulseLines)
                inds.append(i)
        linesPieces = [lines[inds[j]+1:inds[j+1]] + pulseLinesAll[j] for j in range(len(pulseLinesAll))] + [lines[inds[-1]+1:]]
        lines = [line for linesPiece in linesPieces for line in linesPiece]
        return lines

    def getTTLChannels(self):
        TTLChannels = []
        for line in self.lines:
            channel, _ = self.matchTTL(line)
            if (channel):
                if channel not in TTLChannels:
                    TTLChannels.append(channel)
        return TTLChannels
    
    def getDACChannels(self):
        DACChannels = []
        for line in self.lines:
            channel, _ = self.matchDAC(line)
            if (channel):
                if channel not in DACChannels:
                    DACChannels.append(channel)
        return DACChannels
    
    def getSnapshots(self):
        TTLs, DACs, DDSFreqs, DDSAmps = {}, {}, {}, {}
        for channel in self.TTLChannels:
            TTLs[channel] = 0
        for channel in self.DACChannels:
            DACs[channel] = {'start': 0, 'end': 0, 'ramptime': 0}
        for channel in self.DDSFreqChannels:
            DDSFreqs[channel] = {'start': 0, 'end': 0, 'ramptime': 0}
        for channel in self.DDSAmpChannels:
            DDSAmps[channel] = {'start': 0, 'end': 0, 'ramptime': 0}
        t = 0
        for j, line in enumerate(self.lines):
            ttl, value = self.matchTTL(line)
            if (ttl):
                TTLs[ttl] = value
            dac, DACValues = self.matchDAC(line)
            ddsFreq, DDSFreqValues = self.matchDDSFreq(line)
            ddsAmp, DDSAmpValues = self.matchDDSAmp(line)
            if (dac):
                DACs[dac] = DACValues
            if (ddsFreq):
                DDSFreqs[ddsFreq] = DDSFreqValues
            if (ddsAmp):
                DDSAmps[ddsAmp] = DDSAmpValues
            tMatch = re.search("^t\s+[\+=]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
            if (tMatch):
                newSnapshot = Snapshot(t, copy.deepcopy(TTLs), copy.deepcopy(DACs),
                                      copy.deepcopy(DDSFreqs), copy.deepcopy(DDSAmps))
                self.snapshots.append(newSnapshot)
                lineSplit = re.split('\s+', tMatch[0])
                if (lineSplit[1] == '='):
                    t = float(lineSplit[2])
                elif (lineSplit[1] == '+='):
                    t += float(lineSplit[2])
                for dac in self.DACChannels:
                    if (DACs[dac]['ramptime'] != 0):
                        DACs[dac]['start'] = DACs[dac]['end']
                    DACs[dac]['end'] = 0
                    DACs[dac]['ramptime'] = 0
                for ddsFreq in self.DDSFreqChannels:
                    if (DDSFreqs[ddsFreq]['ramptime'] != 0):
                        DDSFreqs[ddsFreq]['start'] = DDSFreqs[ddsFreq]['end']
                    DDSFreqs[ddsFreq]['end'] = 0
                    DDSFreqs[ddsFreq]['ramptime'] = 0
                for ddsAmp in self.DDSAmpChannels:
                    if (DDSAmps[ddsAmp]['ramptime'] != 0):
                        DDSAmps[ddsAmp]['start'] = DDSAmps[ddsAmp]['end']
                    DDSAmps[ddsAmp]['end'] = 0
                    DDSAmps[ddsAmp]['ramptime'] = 0
        
                    
    def matchTTL(self, line):
        onMatch = re.search("^on:\s+[\-a-zA-Z0-9_]+", line)
        offMatch = re.search("^off:\s+[\-a-zA-Z0-9_]+", line)
        channel = ''
        value = 0
        if (onMatch):
            channel = onMatch[0].split(': ')[1]
            value = 1
        elif (offMatch):
            channel = offMatch[0].split(': ')[1]
            value = 0
        return channel, value
    
    def matchDAC(self, line):
        dacMatch = re.search("^dac: [\-a-zA-Z0-9_]+ [\-a-zA-Z0-9_\.]+", line)
        dacRampMatch = re.search("^dacramp: [\-a-zA-Z0-9_]+ [\-a-zA-Z0-9_\.]+ [\-a-zA-Z0-9_\.]+ [\-a-zA-Z0-9_\.]+", line)
        channel = ''
        values = {'start': 0, 'end': 0, 'ramptime': 0}
        if (dacMatch):
            dacMatchSplit = dacMatch[0].split(' ')
            channel = dacMatchSplit[1]
            values['start'] = float(dacMatchSplit[2])
            values['end'] = 0
            values['ramptime'] = 0
        elif (dacRampMatch):
            dacRampMatchSplit = dacRampMatch[0].split(' ')
            channel = dacRampMatchSplit[1]
            values['start'] = float(dacRampMatchSplit[2])
            values['end'] = float(dacRampMatchSplit[3])
            values['ramptime'] = float(dacRampMatchSplit[4])
        return channel, values
    
    def getDACData(self):
        DACData = {}
        for channel in self.DACChannels:
            ts, DACValues = self.getDACChannelValues(self.snapshots, channel)
            DACData[channel] = pd.DataFrame(np.transpose([ts,DACValues]), columns = ['time', 'value'])
        return DACData
    
    def getTTLData(self):
        ts = arr([s.t for s in self.snapshots])
        ts = np.repeat(ts,2)[1:]
        data = []
        data.append(ts)
        for ttl in self.TTLChannels:
            ttlValues = arr([s.TTLChannels[ttl] for s in self.snapshots])
            ttlValues = np.repeat(ttlValues, 2)[:-1]
            data.append(ttlValues)
        data = np.transpose(data)
        TTLData = pd.DataFrame(data, columns = ['time']+[channel for channel in self.TTLChannels])
        return TTLData
    
    def getDACChannelValues(self, snapshots, channel):
        DACValues = [snapshots[0].DACChannels[channel]['start']]
        ts = [0]
        for s in snapshots[1:]:
            cValues = s.DACChannels[channel]
            rt = cValues['ramptime']
            channelChanged = False
            if (DACValues[-1] != cValues['start'] or rt != 0):
                channelChanged = True
            if (channelChanged):
                ts.append(s.t)
                # add point with values before update
                ts.append(s.t)
                DACValues.append(DACValues[-1])
                DACValues.append(cValues['start'])

                if (rt != 0):
                    ts.append(ts[-1]+rt)
                    DACValues.append(cValues['end'])
        if (ts[-1] < snapshots[-1].t):
            DACValues.append(DACValues[-1])
            ts.append(snapshots[-1].t) 

        return ts, DACValues
    
    def getDDSChannels(self):
        DDSFreqChannels = []
        DDSAmpChannels = []
        for line in self.lines:
            channelFreq, _ = self.matchDDSFreq(line)
            channelAmp, _ = self.matchDDSAmp(line)
            if (channelFreq):
                if channelFreq not in DDSFreqChannels:
                    DDSFreqChannels.append(channelFreq)
            if (channelAmp):
                if channelAmp not in DDSAmpChannels:
                    DDSAmpChannels.append(channelAmp)
        return DDSFreqChannels, DDSAmpChannels
    
    def matchDDSFreq(self, line):
        ddsfreqMatch = re.search("^ddsfreq:\s+[\-a-zA-Z0-9_]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
        ddsRampFreqMatch = re.search("^ddsrampfreq:\s+[\-a-zA-Z0-9_]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)\*\/]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
        channel = ''
        values = {'start': 0, 'end': 0, 'ramptime': 0}
        if (ddsfreqMatch):
            ddsMatchSplit = ddsfreqMatch[0].split(' ')
            channel = ddsMatchSplit[1]
            values['start'] = float(ddsMatchSplit[2])
            values['end'] = 0
            values['ramptime'] = 0
        elif (ddsRampFreqMatch):
            ddsRampMatchSplit = ddsRampFreqMatch[0].split(' ')
            channel = ddsRampMatchSplit[1]
            values['start'] = float(ddsRampMatchSplit[2])
            values['end'] = float(ddsRampMatchSplit[3])
            values['ramptime'] = float(ddsRampMatchSplit[4])
        return channel, values
    
    def matchDDSAmp(self, line):
        ddsampMatch = re.search("^ddsamp:\s+[\-a-zA-Z0-9_]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
        ddsRampAmpMatch = re.search("^ddsrampamp:\s+[\-a-zA-Z0-9_]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)\*\/]+\s+[\+\-\*\/a-zA-Z0-9_\.\(\)]+", line)
        channel = ''
        values = {'start': 0, 'end': 0, 'ramptime': 0}
        if (ddsampMatch):
            ddsMatchSplit = ddsampMatch[0].split(' ')
            channel = ddsMatchSplit[1]
            values['start'] = float(ddsMatchSplit[2])
            values['end'] = 0
            values['ramptime'] = 0
        elif (ddsRampAmpMatch):
            ddsRampMatchSplit = ddsRampAmpMatch[0].split(' ')
            channel = ddsRampMatchSplit[1]
            values['start'] = float(ddsRampMatchSplit[2])
            values['end'] = float(ddsRampMatchSplit[3])
            values['ramptime'] = float(ddsRampMatchSplit[4])
        return channel, values

    def getDDSFreqData(self):
        DDSFreqData = {}
        for channel in self.DDSFreqChannels:
            ts, DDSValues = self.getDDSChannelFreqValues(self.snapshots, channel)
            DDSFreqData[channel] = pd.DataFrame(np.transpose([ts,DDSValues]), columns = ['time', 'value'])
        return DDSFreqData

    def getDDSChannelFreqValues(self, snapshots, channel):
        DDSFreqValues = [snapshots[0].DDSFreqChannels[channel]['start']]
        ts = [0]
        for s in snapshots[1:]:
            cValues = s.DDSFreqChannels[channel]
            rt = cValues['ramptime']
            channelChanged = False
            if (DDSFreqValues[-1] != cValues['start'] or rt != 0):
                channelChanged = True
            if (channelChanged):
                ts.append(s.t)
                # add point with values before update
                ts.append(s.t)
                DDSFreqValues.append(DDSFreqValues[-1])
                DDSFreqValues.append(cValues['start'])

                if (rt != 0):
                    ts.append(ts[-1]+rt)
                    DDSFreqValues.append(cValues['end'])
        if (ts[-1] < snapshots[-1].t):
            DDSFreqValues.append(DDSFreqValues[-1])
            ts.append(snapshots[-1].t) 

        return ts, DDSFreqValues

    def getDDSAmpData(self):
        DDSAmpData = {}
        for channel in self.DDSAmpChannels:
            ts, DDSValues = self.getDDSChannelAmpValues(self.snapshots, channel)
            DDSAmpData[channel] = pd.DataFrame(np.transpose([ts,DDSValues]), columns = ['time', 'value'])
        return DDSAmpData

    def getDDSChannelAmpValues(self, snapshots, channel):
        DDSAmpValues = [snapshots[0].DDSAmpChannels[channel]['start']]
        ts = [0]
        for s in snapshots[1:]:
            cValues = s.DDSAmpChannels[channel]
            rt = cValues['ramptime']
            channelChanged = False
            if (DDSAmpValues[-1] != cValues['start'] or rt != 0):
                channelChanged = True
            if (channelChanged):
                ts.append(s.t)
                # add point with values before update
                ts.append(s.t)
                DDSAmpValues.append(DDSAmpValues[-1])
                DDSAmpValues.append(cValues['start'])

                if (rt != 0):
                    ts.append(ts[-1]+rt)
                    DDSAmpValues.append(cValues['end'])
        if (ts[-1] < snapshots[-1].t):
            DDSAmpValues.append(DDSAmpValues[-1])
            ts.append(snapshots[-1].t) 

        return ts, DDSAmpValues

In [6]:
expP = ExperimentPlotter(exp)

# dash test

In [7]:
port = 8060

In [8]:
plotTTLChannels = ['op','rb1','rb2','rb3','imageshutter','cameratrig','clock']
plotDACChannels = ['slmdepth', 'x1coil', 'y1coil', 'z1coil', 'clockpiezohor']
plotDDSChannels = ['dds0', 'dds4']

In [14]:
app = JupyterDash(__name__)

numTTLC = len(plotTTLChannels)
numDACC = len(plotDACChannels)
numDDSC = len(plotDDSChannels)
if (numTTLC>0):
    fig = subplots.make_subplots(rows=numTTLC, cols=1, shared_xaxes=True)
    for i, channel in enumerate(plotTTLChannels):
        fig.add_trace(go.Scatter(x=expP.TTLData['time'], y=expP.TTLData[channel], fill='tozeroy',line=dict(width=0.5, color=cols[0]),
                         mode='lines'), row=i+1, col=1)
        fig.update_yaxes(range=[0, 1], row=i+1, col=1, tickvals=[0.5], ticktext=[channel])
        # fig.update_yaxes(title_text=channel, row=i+1, col=1)
    fig.update_layout(margin=dict(l=5, r=5, t=25, b=5), height=35*numTTLC)
    fig.update_layout(showlegend=False)

if (numDACC>0):
    fig2 = subplots.make_subplots(rows=numDACC, cols=1, shared_xaxes=True)
    for i, channel in enumerate(plotDACChannels):
        fig2.add_trace(go.Scatter(x=expP.DACData[channel]['time'], y=expP.DACData[channel]['value'], 
                                  fill='tozeroy',line=dict(width=0.5, color=cols[0]),
                         mode='lines'), row=i+1, col=1)
        fig2.update_yaxes(row=i+1, col=1, title_text=channel)
    fig2.update_layout(margin=dict(l=5, r=5, t=25, b=5), height=100*numDACC)
    fig2.update_layout(showlegend=False)

if (numDDSC>0):
    fig3 = subplots.make_subplots(rows=numDDSC, cols=1, shared_xaxes=True)
    for i, channel in enumerate(plotDDSChannels):
        fig3.add_trace(go.Scatter(x=expP.DDSFreqData[channel]['time'], y=expP.DDSFreqData[channel]['value'], 
                                  fill='tozeroy', line=dict(width=1, color=cols[0]), mode='lines'), row=i+1, col=1)
        fig3.update_yaxes(row=i+1, col=1, title_text=channel)
    fig3.update_layout(margin=dict(l=5, r=5, t=25, b=5), height=100*numDDSC)
    fig3.update_layout(showlegend=False)

    fig4 = subplots.make_subplots(rows=numDDSC, cols=1, shared_xaxes=True)
    for i, channel in enumerate(plotDDSChannels):
        fig4.add_trace(go.Scatter(x=expP.DDSAmpData[channel]['time'], y=expP.DDSAmpData[channel]['value'], 
                                  fill='tozeroy',line=dict(width=0.5, color=cols[0]),
                         mode='lines'), row=i+1, col=1)
        fig4.update_yaxes(row=i+1, col=1, title_text=channel)
    fig4.update_layout(margin=dict(l=5, r=5, t=25, b=5), height=100*numDDSC)
    fig4.update_layout(showlegend=False)

dcc.Graph(figure=fig)
dcc.Graph(figure=fig2)
dcc.Graph(figure=fig3)
dcc.Graph(figure=fig4)

app.layout = html.Div([
    html.Div([
    html.Div([html.Span(html.B('data address: ')),str(expP.data_addr)+'\data_'+str(expP.file_id)+'.h5  ', html.Br(),
             html.Span(html.B('date/time: ')), expP.experiment_date+' '+expP.experiment_time+'  ', html.Br(),
              html.Span(html.B('key name: ')), expP.key_name+' ', html.Br(),
              html.Span(html.B('variations: ')), str(len(expP.key))+' ', html.Br(),
             ], style={'text-align': 'left', 'font-size': '12px'}),#+' , date:'+expP.experiment_date+' , experiment_time:'+expP.experiment_time+' , key_name:'+expP.key_name+' , reps:'+str(expP.reps)),
    html.Br(),
    html.Div([
        html.Div('time range (ms)', style={'padding-right': '80px'}),
             html.Div(dcc.RangeSlider(
        expP.TTLData['time'].min(),
        expP.TTLData['time'].max(),
        step=None,
        id='time_slider',
        value=[expP.TTLData['time'].min(),expP.TTLData['time'].max()]
        # marks={str(year): str(year) for year in df['Year'].unique()}
    ),style={'padding': '10px'})], style={'width': '90%', 'margin': '0 auto'})
    ], style={'position': 'fixed', 'top': '0', 'z-index': '100', 
              'background-color': '#fff',
             'padding': '20px 60px 20px 20px', 
              'width': '100%', 'border': '1px solid #ccc'}),
    html.Div([
    html.Div([
        # html.Div('test'),
        html.Div('TTL channels', style={'clear': 'both'})
    ], style={'padding-top': '200px'}),
    html.Div([
        dcc.Graph(figure=fig, id='ttl_plots'),
    ]),
    html.Br(),
    html.Div('DAC channels', style={'clear': 'both'}),
    html.Div([
        dcc.Graph(figure=fig2, id='dac_plots')
    ], style={'padding-left': '27px'}),
    html.Br(),
    html.Div('DDS channels (frequency)', style={'clear': 'both'}),
    html.Div([
        dcc.Graph(figure=fig3, id='dds_freq_plots')
    ], style={'padding-left': '27px'}),
    html.Br(),
    html.Div('DDS channels (amplitude)', style={'clear': 'both'}),
    html.Div([
        dcc.Graph(figure=fig4, id='dds_amp_plots')
    ], style={'padding-left': '27px'}),
    html.Br(),
    html.Br(),
    html.Br(),
    html.Br(),
    html.Br(),
], style={'margin': '20px 80px 0px 40px'})
    ], style={'padding': '0', 'margin': '0', 'font-family': 'sans-serif', 'text-align': 'center'})

@app.callback(
    [Output('ttl_plots', 'figure'),
    Output('dac_plots', 'figure'),
    Output('dds_freq_plots', 'figure'),
    Output('dds_amp_plots', 'figure')],
    Input('time_slider', 'value'),
    State('ttl_plots', 'figure'),
    State('dac_plots', 'figure'),
    State('dds_freq_plots', 'figure'),
    State('dds_amp_plots', 'figure'))
def update_plots(timeValue, figTTL, figDAC, figDDSFreq, figDDSAmp):
    f1 = go.Figure(figTTL)
    f1.update_xaxes({'range': timeValue, 'autorange': False})
    f2 = go.Figure(figDAC)
    f2.update_xaxes({'range': timeValue, 'autorange': False})
    f3 = go.Figure(figDDSFreq)
    f3.update_xaxes({'range': timeValue, 'autorange': False})
    f4 = go.Figure(figDDSAmp)
    f4.update_xaxes({'range': timeValue, 'autorange': False})
    return f1, f2, f3, f4

app.run_server(debug=True, mode='external')

Dash app running on http://127.0.0.1:8050/
